## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup (added numpy)
import pandas as pd
import requests
import gmaps
import numpy as np
from ipywidgets.embed import embed_minimal_html

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/jmorales/Desktop/CLASSWORK/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Cape Town,-33.9258,18.4232,74.26,54,0,14.97,clear sky,ZA
1,1,Bredasdorp,-34.5322,20.0403,61.68,81,13,22.30,few clouds,ZA
2,2,Busselton,-33.6500,115.3333,60.01,60,96,7.23,overcast clouds,AU
3,3,Diego De Almagro,-26.3667,-70.0500,77.25,18,20,8.46,few clouds,CL
4,4,Paamiut,61.9940,-49.6678,29.17,58,1,7.56,clear sky,GL


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter minium temperature for your vacation "))
max_temp = float(input("Please enter your maximum temperature for your vacation "))

Please enter minium temperature for your vacation 65
Please enter your maximum temperature for your vacation 99


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
user_preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Cape Town,-33.9258,18.4232,74.26,54,0,14.97,clear sky,ZA
3,3,Diego De Almagro,-26.3667,-70.0500,77.25,18,20,8.46,few clouds,CL
7,7,Aracaju,-10.9111,-37.0717,87.75,66,40,12.66,scattered clouds,BR
9,9,Takoradi,4.8845,-1.7554,83.88,75,91,13.62,light rain,GH
11,11,Bandarbeyla,9.4942,50.8122,81.16,77,99,9.91,overcast clouds,SO
13,13,Castro,-24.7911,-50.0119,78.48,56,66,6.89,broken clouds,BR
14,14,Itarema,-2.9248,-39.9167,78.62,83,62,11.36,broken clouds,BR
16,16,Hun,29.1268,15.9477,80.47,18,18,11.48,few clouds,LY
17,17,Nanyamba,-10.6833,39.8333,71.35,97,9,3.42,clear sky,TZ
18,18,Korla,41.7597,86.1469,70.32,19,86,9.80,overcast clouds,CN


In [5]:
# 4a. Determine if there are any empty rows.
user_preferred_cities_df.isna().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                7
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## I have to drop 7 Country Rows according to the above

clean_user_pref_cities_df = user_preferred_cities_df.dropna()
clean_user_pref_cities_df.isna().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_user_pref_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,74.26,clear sky,-33.9258,18.4232,
3,Diego De Almagro,CL,77.25,few clouds,-26.3667,-70.0500,
7,Aracaju,BR,87.75,scattered clouds,-10.9111,-37.0717,
9,Takoradi,GH,83.88,light rain,4.8845,-1.7554,
11,Bandarbeyla,SO,81.16,overcast clouds,9.4942,50.8122,
13,Castro,BR,78.48,broken clouds,-24.7911,-50.0119,
14,Itarema,BR,78.62,broken clouds,-2.9248,-39.9167,
16,Hun,LY,80.47,few clouds,29.1268,15.9477,
17,Nanyamba,TZ,71.35,clear sky,-10.6833,39.8333,
18,Korla,CN,70.32,overcast clouds,41.7597,86.1469,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,74.26,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
3,Diego De Almagro,CL,77.25,few clouds,-26.3667,-70.0500,hotel luna del desierto
7,Aracaju,BR,87.75,scattered clouds,-10.9111,-37.0717,HOTEL FÊNIX
9,Takoradi,GH,83.88,light rain,4.8845,-1.7554,Raybow International Hotel
11,Bandarbeyla,SO,81.16,overcast clouds,9.4942,50.8122,JABIR HOTEL
13,Castro,BR,78.48,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
14,Itarema,BR,78.62,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
16,Hun,LY,80.47,few clouds,29.1268,15.9477,Alrwasi Hotel
17,Nanyamba,TZ,71.35,clear sky,-10.6833,39.8333,Busati Lodge Nanyamba
18,Korla,CN,70.32,overcast clouds,41.7597,86.1469,Gulisitan Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna() - This did not work since there are blanks, not NAs
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace('',np.nan,regex = True) #replaced blanks
clean_hotel_df = hotel_df.dropna() #now we can drop NAs
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,74.26,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
3,Diego De Almagro,CL,77.25,few clouds,-26.3667,-70.0500,hotel luna del desierto
7,Aracaju,BR,87.75,scattered clouds,-10.9111,-37.0717,HOTEL FÊNIX
9,Takoradi,GH,83.88,light rain,4.8845,-1.7554,Raybow International Hotel
11,Bandarbeyla,SO,81.16,overcast clouds,9.4942,50.8122,JABIR HOTEL
13,Castro,BR,78.48,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
14,Itarema,BR,78.62,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
16,Hun,LY,80.47,few clouds,29.1268,15.9477,Alrwasi Hotel
17,Nanyamba,TZ,71.35,clear sky,-10.6833,39.8333,Busati Lodge Nanyamba
18,Korla,CN,70.32,overcast clouds,41.7597,86.1469,Gulisitan Hotel


In [ ]:
# 8a. Create the output File (CSV)
output_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#embed_minimal_html('export.html', views=[fig])

In [ ]:
#NOTE: For some reason, the map would not display. 
## Methods taken to remediate: 
### I reinstalled gmaps using my terminal AND the pip install methods. 
### I also created a new gmaps API key and pasted it into my config file
#### I contacted AskBCS for assistance
#### Exported figure to html using "embed_minimal_html." SUCCESS! Filed saved in folder as html. 

#UPDATE!!! REINSTALLED GMAPS AGAIN AND RESTARTED COMPUTER: FINALLY WORKED AND SHOWS IN JUPYTER NOTEBOOK